<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn;">

<p>در ASP.NET Core، الگوی <strong>Options</strong> برای مدیریت و دسترسی به تنظیمات پیکربندی (configuration settings) به صورت قوی‌نوع (strongly-typed) استفاده می‌شود. این الگو به شما این امکان را می‌دهد که تنظیمات پیکربندی برنامه را به طور مستقیم به اشیاء POCO اختصاص دهید و سپس در سرویس‌ها و کنترلرها از آن‌ها استفاده کنید. این روش باعث می‌شود که مدیریت پیکربندی‌ها ساده‌تر، قابل تست‌تر و قابل مدیریت‌تر شود.</p>

<h5>مزایای استفاده از Options Pattern</h5>
<ol><li><p><strong>Strongly Typed:</strong> استفاده از کلاس‌های strongly-typed به شما اطمینان می‌دهد که تنظیمات شما به صورت صحیح مدیریت می‌شوند و در زمان کامپایل، خطاهای احتمالی شناسایی می‌شوند.</p></li><li><p><strong>قابل تست بودن:</strong> این الگو به راحتی قابل تست است، زیرا تنظیمات به عنوان DI در سرویس‌ها و کنترلرها تزریق می‌شوند.</p></li><li><p><strong>جداسازی concerns:</strong> تنظیمات به طور جدی از سایر قسمت‌های برنامه جدا می‌شوند، که این امر نگهداری و مدیریت آن‌ها را آسان‌تر می‌کند.</p></li><li><p><strong>پشتیبانی از تغییرات داینامیک:</strong> با استفاده از الگوهای پیشرفته‌تر مانند <code>IOptionsSnapshot</code>، می‌توانید از تغییرات داینامیک در تنظیمات خود پشتیبانی کنید.</p></li></ol>
</div>

In [ ]:
{
    "MapSettings": { 
        "DefaultZoomLevel": 6, 
        "DefaultLocation": { 
            "latitude": 50.500, 
            "longitude": -4.000 
        }
    },
    "AppDisplaySettings": { 
        "Title": "Acme Store Locator", 
        "ShowCopyright": true 
    }
}

In [ ]:
app.MapGet("/display-settings", (Iconfiguration config) =>
{
    string title = config["AppDisplaySettings:Title"];

    bool showCopyright = bool.Parse(
        config["AppDisplaySettings:ShowCopyright"]);
        
    return new { title, showCopyright };
});

In [ ]:
app.MapGet("/display-settings",
    (IOptions<AppDisplaySettings> options) => 
{
    AppDisplaySettings settings = options.Value; 

    string title = settings.Title; 

    bool showCopyright = settings.ShowCopyright; 
    
    return new { title, showCopyright };
});

The ASP.NET Core ***configuration system*** includes a `binder`, which can **take** a
collection of ***configuration values*** and bind them to a ***strongly typed object***

### Introducing the IOptions interface

In [ ]:
public class AppDisplaySettings
{
    public string Title { get; set; }
    public bool ShowCopyright { get; set; }
}

Your ***options classes*** need to be `nonabstract` and have a `public parameterless
constructor` to be eligible for binding.

you can use nested complex types, The ***options system*** `binds` sections to complex

ASP.NET Core introduces the `IOptions<T>` interface, ***binding*** of configuration values to your custom `POCO options classes`, have a single property, `Value`

In [ ]:
WebApplicationBuilder builder = WebApplication.CreateBuilder(args);

builder.Services.Configure<MapSettings>(
    builder.Configuration.GetSection("MapSettings"));

builder.Services.Configure<AppDisplaySettings>(
    builder.Configuration.GetSection("AppDisplaySettings"));

The binding of the `T` options class to ConfigurationSection happens when
you `first request IOptions<T>`.

This setup has ***one catch***: `you can’t use the reloadOnChange` parameter

### Reloading strongly typed options with `IOptionsSnapshot`

you can use the `IOptionsSnapshot<T>` interface, for get `fresh data` when ***source*** of configuration is ***changed***

`IOptionsSnapshot<T>` is registered as a `scoped service`, so `you can’t inject` it
into ***singleton services***, if you do, you’ll have a `captive dependency`  
  
If you need a ***singleton*** version of `IOptionsSnapshot<T>`, you can use a similar interface, `IOptionsMonitor<T>`.


`IOptionsSnapshot<T>` ***re-create*** at `everey request`. If you edit the settings file and cause IConfiguration to reload, `IOptionsSnapshot<T>` shows the new values on the ***next request***.

### Configuring strongly typed settings without IOptions

In [ ]:
WebApplicationBuilder builder = WebApplication.CreateBuilder(args);

var settings = new MapSettings (); 

builder.Configuration.GetSection("MapSettings").Bind(settings); 

builder.Services.AddSingleton(settings); 

WebApplication app = builder.Build();

app.MapGet("/", (MapSettings mapSettings) => mapSettings); 

app.Run();

In [ ]:
WebApplicationBuilder builder = WebApplication.CreateBuilder(args);

builder.Services.Configure<MapSettings>( 
    builder.Configuration.GetSection("MapSettings")); 
    
builder.Services.AddSingleton(provider => 
    provider.GetRequiredService<IOptions<MapSettings>>().Value); 

WebApplication app = builder.Build();

app.MapGet("/", (MapSettings mapSettings) => mapSettings); 

app.Run();